## Spark streaming test

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_producer import *
from kafka import KafkaConsumer

subreddit = "technology"
print("imported modules")

imported modules


In [4]:
spark = SparkSession.builder.appName("reddit_" + subreddit) \
            .master("spark://spark-master:7077") \
            .config("spark.eventLog.enabled", "true") \
            .config("spark.eventLog.dir", "file:///opt/workspace/events") \
            .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0") \
            .getOrCreate()

spark.sparkContext.setLogLevel('WARN')
print("created spark successfully")

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-04e84e97-7c4a-474c-a813-ea580c16f129;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in local-m2-cache
downloading file:/root/.m2/repository/org/apache/hadoop/hadoop-aws/3.2.0/hadoop-aws-3.2.0.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.2.0!hadoop-aws.jar (7ms)
downloading file:/root/.m2/repository/com/amazonaws/aws-java-sdk-bundle/1.11.375/aws-java-sdk-bundle-1.11.375.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.11.375!aws-java-sdk-bundle.jar (799ms)
:: resolution report :: resolve 2300ms :: artifacts dl 813ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from local-m2-cache in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from local-m2-cache in [def

created spark successfully


In [3]:
test_df = spark \
            .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "kafka:9092") \
                .option("subscribe", "reddit_" + subreddit) \
                .option("startingOffsets", "earliest") \
                .load() \
                .selectExpr("CAST(value AS STRING)")

test_df.writeStream \
    .trigger(processingTime='5 seconds') \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "true") \
    .start() \
    .awaitTermination()

22/05/06 19:27:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-69225490-358e-48d3-8f12-3bd0029eb3be. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/06 19:27:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"kind": "Listing...|
|{"kind": "Listing...|
|{"kind": "Listing...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
|{"approved_at_utc...|
+--------------------+
only showing top 20 rows



22/05/06 19:27:41 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5644 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



22/05/06 20:57:13 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 8735 milliseconds


-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



KeyboardInterrupt: 

In [5]:
spark.stop()
print("stopped spark successfully.")